In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import load
from sklearn.inspection import permutation_importance
import shap

In [2]:
# Load data and model
cwd = os.getcwd()
df_lr_p = pd.read_parquet(os.path.join(cwd, 'data', 'modelling', 'lgbm2.parquet'))
df_lr.sort_index(inplace=True)
df_lgbm = pd.read_parquet(os.path.join(cwd, 'data', 'modelling', 'lgbm2.parquet'))
df_lgbm.sort_index(inplace=True)
df_X = df_lgbm.drop(['goal', 'split', 'match_id', 'wyscout_id', 'statsbomb_id'], axis=1).copy()
model = load(os.path.join(cwd, 'models', 'lgbm_model.joblib'))

In [3]:
# Get feature names
features = df_X.columns
features = np.array([f.replace('_', ' ') for f in features])
features

array(['shot type name', 'counter attack', 'fast break', 'strong foot',
       'body part name', 'assist type', 'pass end y', 'pass end x',
       'pass switch', 'pass cross', 'pass cut back', 'pass height name',
       'pass technique name', 'carry length', 'visible angle',
       'middle angle', 'distance to goal', 'distance visible angle',
       'log distance to goal', 'shot one on one', 'shot open goal',
       'under pressure', 'match week', 'H A column', 'players',
       'players rival', 'area shot', 'area goal', 'n angle',
       'goalkeeper x', 'goalkeeper y', 'smart pass', 'match moment',
       'shot number', 'shot zone number', 'shot player number',
       'shot zone player number'], dtype='<U23')

In [4]:
# xg predictions. First have to fit to training data
df_lgbm['xg'] = model.predict_proba(df_X)[:, 1]
df_lgbm

,shot_type_name,counter_attack,fast_break,strong_foot,body_part_name,assist_type,pass_end_y,pass_end_x,pass_switch,pass_cross,...,shot_number,shot_zone_number,shot_player_number,shot_zone_player_number,goal,split,match_id,wyscout_id,statsbomb_id,xg
5,4,False,False,False,1,0,39.61,91.9625,0.0,0.0,...,1.0,1.0,1.0,1.0,False,train,2275099,NaN,4f985308-bf76-4a5d-860c-93537b4a49e3,0.081755
6,4,False,False,True,0,1,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,False,train,7471,NaN,7e68fa7c-4e44-4b3a-ab8a-9ff1da69eacc,0.020736
7,4,False,False,False,1,1,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,False,test,7585,NaN,38760bf6-275c-4d8a-aac4-5ebfb570891b,0.049965
8,4,False,False,True,0,0,30.60,77.0000,0.0,0.0,...,1.0,1.0,1.0,1.0,False,train,19788,NaN,bfb97b05-b013-4344-83a7-da6e19b57c05,0.017617
9,4,False,False,True,0,1,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,False,train,7571,NaN,25f7433a-e83b-4960-a619-7c991b872952,0.027208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65782,4,False,False,True,1,0,NaN,NaN,0.0,0.0,...,11.0,1.0,1.0,1.0,True,test,2516750,182026555.0,None,0.728327
65783,2,False,False,False,1,1,NaN,NaN,NaN,NaN,...,9.0,4.0,1.0,1.0,False,test,2499739,182137485.0,None,0.033731
65785,4,False,False,False,1,0,NaN,NaN,0.0,0.0,...,17.0,3.0,2.0,1.0,False,train,2516750,182026562.0,None,0.108924
65786,4,False,True,True,0,0,NaN,NaN,0.0,0.0,...,9.0,5.0,2.0,1.0,False,train,2499820,203362910.0,None,0.098818


In [5]:
# Get a datafrane if the uncalibrated/ calibrated probabilities for each of the three models
models = model.calibrated_classifiers_
estimators = [model.base_estimator for model in models]
probabilities = []
for i in range(3):
    probabilities.append(estimators[i].predict_proba(df_X)[:, 1])
    probabilities.append(models[i].predict_proba(df_X)[:, 1])
df_probabilities = pd.DataFrame(np.vstack(probabilities).T, columns=['uncalibrated0', 'calibrated0',
                                                                     'uncalibrated1', 'calibrated1',
                                                                     'uncalibrated2', 'calibrated2'])

df_probabilities['calibrated'] = model.predict_proba(df_X)[:, 1]
df_probabilities

,uncalibrated0,calibrated0,uncalibrated1,calibrated1,uncalibrated2,calibrated2,calibrated
0,0.104575,0.090823,0.089587,0.080895,0.086232,0.073548,0.081755
1,0.025714,0.017427,0.025361,0.024217,0.026424,0.020563,0.020736
2,0.053393,0.062078,0.045585,0.046901,0.043557,0.040915,0.049965
3,0.022093,0.008070,0.021738,0.024217,0.027478,0.020563,0.017617
4,0.039639,0.024502,0.035504,0.028604,0.034260,0.028517,0.027208
...,...,...,...,...,...,...,...
64658,0.702410,0.698113,0.657693,0.756098,0.666114,0.730769,0.728327
64659,0.036553,0.019417,0.038165,0.045045,0.041343,0.036730,0.033731
64660,0.111482,0.095133,0.129090,0.112554,0.125285,0.119086,0.108924
64661,0.100062,0.090823,0.099583,0.098430,0.107313,0.107203,0.098818


In [6]:
probabilities

[array([0.10457527, 0.02571357, 0.0533933 , ..., 0.11148163, 0.10006222,
        0.08853809]),
 array([0.09082263, 0.01742739, 0.06207827, ..., 0.09513274, 0.09082263,
        0.09082263]),
 array([0.08958719, 0.02536139, 0.04558479, ..., 0.12908996, 0.09958251,
        0.09963955]),
 array([0.08089501, 0.02421652, 0.04690117, ..., 0.11255411, 0.09842995,
        0.09842995]),
 array([0.08623186, 0.02642432, 0.04355736, ..., 0.12528528, 0.10731313,
        0.07957593]),
 array([0.07354839, 0.02056325, 0.04091456, ..., 0.11908646, 0.10720268,
        0.07242063])]

In [7]:
# Shap values from shap package (estimate as probabilites rather than log odds deviation from bias). 
# Done in loop for the estimators in the calibrated classifier

# this takes a while, but calculates the contributions as probabilities
# https://github.com/slundberg/shap/issues/963
sample = data=df_X.sample(500).astype(np.float32)
contributions = []
for estimator in estimators:
    explainer = shap.TreeExplainer(estimator, data=sample, model_output='probability')
    shap_values_probability = explainer.shap_values(df_X)
    bias = explainer.expected_value
    df_contributions_probability = pd.DataFrame(shap_values_probability, columns=features)
    df_contributions_probability['bias'] = bias
    contributions.append(df_contributions_probability)

100%|===================| 64346/64663 [02:00<00:00]        

In [8]:
contributions

[       shot type name  counter attack  fast break  strong foot  \
 0            0.008454       -0.002331         0.0    -0.005901   
 1            0.004017       -0.001749         0.0     0.002272   
 2            0.005786       -0.002160         0.0    -0.004913   
 3            0.003798       -0.001542         0.0     0.002162   
 4            0.005119       -0.001976         0.0     0.002342   
 ...               ...             ...         ...          ...   
 64658        0.008318       -0.004582         0.0     0.006687   
 64659       -0.007158       -0.002798         0.0    -0.003809   
 64660        0.008706       -0.003936         0.0    -0.006071   
 64661        0.008286       -0.003607         0.0     0.003332   
 64662        0.007210       -0.003537         0.0     0.003568   
 
        body part name  assist type  pass end y  pass end x  pass switch  \
 0            0.008731    -0.000887   -0.004970   -0.000109          0.0   
 1            0.003445    -0.001042    0.0

In [9]:
# Scale the uncalibrated contributions to sum to the calibrated predicted probability
scaled_contributions = []
for i in range(3):
    scaled = (contributions[i]
              .divide(df_probabilities[f'uncalibrated{i}'], axis=0)
              .multiply(df_probabilities[f'calibrated{i}'], axis=0))
    scaled_contributions.append(scaled)   
df_scaled_contributions = (scaled_contributions[0] + scaled_contributions[1] + scaled_contributions[2])/3.

In [10]:
# Remove a few where the scaled contributions don't match the actual xg
df_base = df_lgbm[['match_id', 'wyscout_id', 'statsbomb_id', 'xg']]
print(df_scaled_contributions.shape)
print(df_base.shape)

(64663, 38)
(64663, 4)


In [11]:
# Where scaled contributions not within 5 decimal places of the Xg value set to missing (some rounding errors)
df_scaled_contributions.reset_index(drop=True, inplace=True)
df_base.reset_index(drop=True, inplace=True)

In [12]:
mask_contributions_not_equal = ((df_scaled_contributions.sum(axis=1) - df_base.xg).abs().round(5) > 0)
df_scaled_contributions[mask_contributions_not_equal] = np.nan

In [13]:
# Return a dataframe with the contirubtions, match and event ids, and the contributions and xg
df_base = df_base.merge(df_scaled_contributions,
                        left_index=True,
                        right_index=True,
                        how='left',
                        validate='1:1')

In [14]:
df_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64663 entries, 0 to 64662
Data columns (total 42 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   match_id                 64663 non-null  int64  
 1   wyscout_id               42702 non-null  float64
 2   statsbomb_id             21961 non-null  object 
 3   xg                       64663 non-null  float64
 4   shot type name           63895 non-null  float64
 5   counter attack           63895 non-null  float64
 6   fast break               63895 non-null  float64
 7   strong foot              63895 non-null  float64
 8   body part name           63895 non-null  float64
 9   assist type              63895 non-null  float64
 10  pass end y               63895 non-null  float64
 11  pass end x               63895 non-null  float64
 12  pass switch              63895 non-null  float64
 13  pass cross               63895 non-null  float64
 14  pass cut back         

In [15]:
# Save the dataframe
df_base.to_parquet(os.path.join(cwd, 'data', 'modelling', 'xg_shap2.parquet'))

In [16]:
# Use Shap to plot a contribution
idx=2
shap.force_plot(df_scaled_contributions.iloc[idx, -1], 
                df_scaled_contributions.iloc[idx, :-1].values,
                features=df_X.iloc[idx].values,
                feature_names=features, matplotlib=True, show=False)
fig = plt.gcf()
fig.savefig(os.path.join(cwd, 'figures', '08_shap_example.png'), bbox_inches = 'tight', pad_inches = 0.2)